In [248]:
import math as m
import pandas as pd

def d(deg,mnt=0,sec=0):
    """
    Enter the degree, minute, second angle
    returns the angle in radians
    ____
    Overloaded function
    Enter the degree, minute, second angle
    returns the angle in radians
    """
    if type(deg) == type([]):
        return m.radians(deg[0] + deg[1]/60 + deg[2]/3600)
    else:
        return m.radians(deg + mnt/60 + sec/3600)
        
    
def dms(r):
        """
        Takes in angle in radians
        returns andgle in degree, minute, second
        """
        dd = m.degrees(r)
        deg = m.floor(dd)
        
        mm = (dd - deg)/(1/60)
        mnt = m.floor(mm)
        
        ss = (mm - mnt)/(1/60)
        sec = m.floor(ss)

        return [deg,mnt,ss]
    
        
class Point():
    """
    Contains the values that built this point
    """
    def __init__(self, E, N, name="N/A", distance = 0, angle = 0):
        """
        Description:
        
        Input:
            E,
            N,
            name, of the point
            distance, distance used from the previous point 
                        to this point to calculate linear misclosure
        Output:
        
        """
        self.angle = angle
        self.distance = distance
        self.E = E
        self.N = N
        self.name = name
        
    def output(self):
        """
        Outputs the values of the point in a clean format
        """
        print(self.name + ", E: " + str(self.E) + " N: " + str(self.N))
        
        
class Traverse():
    """
    """
    
    def __init__(self, controlA, controlB, angleB, distance):
        """
        Definition:
            Established the bearing to the first point via the first two control points
        
        Input:
            controlA Point()
            controlB Point()
            angleB to first point in traverse (dms) formatted as [degrees, minutes, seconds]
            distance to first point in traverse
            
        Output:
            self.cA 
            self.cB 
            self.aB (radians)
            self.d (meters)
        
        """
        self.cA = controlA
        self.cB = controlB
        self.aB = d(angleB[0],angleB[1],angleB[2])
        self.d = distance
        
        
        self.pointList = [self.cA, self.cB]
        
        #initialize first point
        self.add_point(name = '50')
        
    def bearing(self):
        """
        Definition:
            Calculated the bearing from controlA to controlB
        
        Input:
            
        Output:
            returns the bearing in radians
        """
        deltaE = self.pointList[-1].E - self.pointList[-2].E
        deltaN = self.pointList[-1].N - self.pointList[-2].N

        #in radians
        bearing = m.atan2(deltaE, deltaN)

        return bearing - self.aB + m.pi
    
        if len(self.pointList) <= 2:
            #then this is the first point being initialized and the two points in it are both control points
            deltaE = self.cB.E - self.cA.E
            deltaN = self.cB.N - self.cA.N

            #in radians
            bearing = m.atan2(deltaE, deltaN)

            return bearing - self.aB + m.pi
        
        
    def add_point(self, angle=0, distance=0, name = 'N/A', north = "notgiven"):
        """
        Definition:
            Adds the next point in the traverse list
        
        Input:
            
        Output:
        """
        if north == "given":
            b = d(angle)
            N = self.pointList[-1].N + distance*m.cos(b)
            E = self.pointList[-1].E + distance*m.sin(b)

            point = Point(E, N, name = name, distance = distance, angle = angle)

            self.pointList.append(point)
            
        elif angle == 0 and distance == 0:
            #this means its our first non control point
            b = self.bearing()
            N = self.pointList[-1].N + self.d*m.cos(b)
            E = self.pointList[-1].E + self.d*m.sin(b)

            point = Point(E,N, name = name, distance = self.d, angle = dms(self.aB))

            self.pointList.append(point)
        
        else:
            self.aB = d(angle[0],angle[1],angle[2])
            
            b = self.bearing()
            N = self.pointList[-1].N + distance*m.cos(b)
            E = self.pointList[-1].E + distance*m.sin(b)

            point = Point(E, N, name = name, distance = distance, angle = angle)

            self.pointList.append(point)
            
    def print_points(self):
        """
        Returns a dataframe of all points in the order that they were added in the traverse
        """
        df = pd.DataFrame(columns = ["Point", "Easting", "Northing", "Distance", "Angle"])
        for point in self.pointList:
            #point.output()
            df.loc[len(df.index)] = [point.name, point.E, point.N, point.distance, point.angle]
            
        df = df.set_index("Point")
        
        return df
    
    def calc_linear_misclosure(self, c):
        """
        Calculates what the linear mislosure is.
        Assumes that the final point is the control point in the traverse used to
        calculate linear mislosure
        
        
        c: control point c
        """
        miscE = c.E - self.pointList[-1].E
        miscN = c.N - self.pointList[-1].N
        
        misc = m.sqrt(miscE**2 + miscN**2)
        
        self.misc = misc
        self.miscE = miscE
        self.miscN = miscN
        print(self.miscE)
        print(self.miscN)
    
    def apply_linear_miscolsure(self):
        """
        Applies linear misclosure to all points
        
        Assumes first point to apply linear miscolsure to is in index 2 (the third point)
        """
        i = 2
        total = self.start_to_point(len(self.pointList)-1)
        
        while i < len(self.pointList):
            
            self.pointList[i].E = (i-1) / (len(self.pointList) - 2)* self.miscE + self.pointList[i].E
            self.pointList[i].N = (i-1) / (len(self.pointList) - 2)* self.miscN + self.pointList[i].N
    
            i = i + 1
                      
    def calc_angle_misclosure(self, c, d):
        """
        Calculates the angular misclosure of the final point
        
        via c being the true final point
        and d being the next control point
        """
        deltaE = c.E - d.E
        deltaN = c.N - d.N

        #in radians
        bearingT = m.atan2(deltaE, deltaN)

        deltaE = self.pointList[-1].E - d.E
        deltaN = self.pointList[-1].N - d.N

        #in radians
        bearing = m.atan2(deltaE, deltaN)
        
        self.angle_misc = dms(bearing - bearingT)
        
        return self.angle_misc
    
    def apply_angle_misclosure(self):
        """
        Applys the angular misclosure
        """
        i = 2
        total = self.start_to_point(len(self.pointList)-1)
        count = 0
        
        tempPoints = self.pointList
        self.pointList = self.pointList[0:2]
        
        print(self.angle_misc)
        
        while i < len(tempPoints):
            
            #angle = dms((i-1) / (len(tempPoints) - 2) * d(self.angle_misc) + d(tempPoints[i].angle))
            angle = dms(1 / (len(tempPoints) - 2) * d(self.angle_misc) + d(tempPoints[i].angle))
            self.add_point(angle = angle, distance = tempPoints[i].distance, name = tempPoints[i].name)
        
            i = i + 1
            
        print(self.pointList[-1].name)
        
    def start_to_point(self, i):
        """
        Calculated the total distance measured to the index of the point for index 1
        
        Assumes index is in range of pointList
        """
        distance = 0
        count = 1
        while count <= i:
            distance = distance + self.pointList[count].distance
            
            count = count + 1
            
        return distance

In [254]:
C35782 = Point(77761.853, 6118935.538, 'ASCM 35782')
C62596 = Point(77958.809, 6118906.918, 'ASCM 62596')

#50
trav1 = Traverse(C35782, C62596, [246,59,22], 531.486)

#511
trav1.add_point(angle = [170, 19, 53], distance = 218.312, name = '511')

df = trav1.print_points()

#51
trav1.add_point(angle = [119,6,2], distance = 136.255, name = '51')

#52
trav1.add_point(angle = [161,25,48], distance = 160.852, name = '52')

#53
trav1.add_point(angle = [254,22,29], distance = 81.123, name = '53')

#54
trav1.add_point(angle = [143,20,0], distance = 104.838, name = '54')

#40
trav1.add_point(angle = [144,0,28], distance = 64.425, name = '40')

#ASCM 260869
trav1.add_point(angle = [224,33,25], distance = 332.526, name = 'ASCM 260869')

#print(trav.print_points())

C137364 = Point(N = 6118844.349, E = 79177.981, name = "ASCM 137364")
C260869 = Point(N = 6119546.073, E = 79188.509, name = "ASCM 260869")

trav1.calc_angle_misclosure(C260869, C137364)
trav1.apply_angle_misclosure()

trav1.calc_linear_misclosure(C260869)
trav1.apply_linear_miscolsure()

trav1.print_points()

#trav1.pointList[3].output()

,Easting,Northing,Distance,Angle
Point,,,,
ASCM 35782,77761.853000,6.118936e+06,0,0
ASCM 62596,77958.809000,6.118907e+06,0,0
50,78234.755150,6.119361e+06,531.486,"[246, 59, 22.000000000000455]"
511,78377.828357,6.119526e+06,218.312,"[170, 19, 53]"
51,78511.181283,6.119498e+06,136.255,"[119, 6, 2]"
52,78649.894906,6.119417e+06,160.852,"[161, 25, 48]"
53,78708.290969,6.119473e+06,81.123,"[254, 22, 29]"
54,78812.281157,6.119486e+06,104.838,"[143, 20, 0]"
40,78868.790947,6.119455e+06,64.425,"[144, 0, 28]"


In [102]:
#50
trav = Traverse(C35782, C62596, [246,59,22], 531.486)

#57
trav.add_point(angle = [127,11,25], distance = 222.952, name = "57")

#51
trav.add_point(angle = [238,27,50], distance = 126.390, name = "51")

#511
trav.add_point(angle = [283,46,41], distance = 136.255, name = "511")

trav.print_points()

,Easting,Northing
Point,,
ASCM 35782,77761.853000,6.118936e+06
ASCM 62596,77958.809000,6.118907e+06
50,78234.755150,6.119361e+06
57,78456.521389,6.119384e+06
51,78511.180971,6.119498e+06
511,78377.827910,6.119526e+06


In [252]:
C137364 = Point(N = 6118844.349, E = 79177.981, name = "ASCM 137364")
C260869 = Point(N = 6119546.073, E = 79188.509, name = "ASCM 260869")

#40
trav = Traverse(C137364, C260869, dms(d([360,0,0]) - d([73,17,9])), 332.526)
trav.pointList[2].name = "40"

#54
trav.add_point(angle = dms(d([360,0,0]) - d([224,33,25])), distance = 64.425, name = '54')

#53
trav.add_point(angle = dms(d([360,0,0]) - d([144,0,28])), distance = 104.838, name = "53")

#52
trav.add_point(angle = dms(d([360,0,0]) - d([143,20,0])), distance = 81.123, name = "52")

#51
trav.add_point(angle = dms(d([360,0,0]) - d([254,22,29])), distance = 160.852, name = "51")

#511
trav.add_point(angle = dms(d([360,0,0]) - d([161,25,48])), distance = 136.255, name = "511")

trav.print_points()

,Easting,Northing,Distance,Angle
Point,,,,
ASCM 137364,79177.981000,6.118844e+06,0,0
ASCM 260869,79188.509000,6.119546e+06,0,0
40,78868.632956,6.119455e+06,332.526,"[286, 42, 50.99999999991269]"
54,78812.124070,6.119486e+06,64.425,"[135, 26, 34.9999999999136]"
53,78708.133493,6.119473e+06,104.838,"[215, 59, 31.999999999957254]"
52,78649.735787,6.119417e+06,81.123,"[216, 39, 59.999999999965894]"
51,78511.024541,6.119498e+06,160.852,"[105, 37, 30.999999999913825]"
511,78377.672432,6.119526e+06,136.255,"[198, 34, 12.000000000077762]"


In [253]:
#calculating 711
trav1.pointList = trav1.pointList[0:4]
trav1.add_point(angle = [131,59,39], distance = 407.619, name = "702")

trav1.pointList[4].output()
trav1.print_points()

702, E: 78785.32740216645 N: 6119533.457896329


,Easting,Northing,Distance,Angle
Point,,,,
ASCM 35782,77761.853000,6.118936e+06,0,0
ASCM 62596,77958.809000,6.118907e+06,0,0
50,78234.727557,6.119361e+06,531.486,"[246, 59, 27.650493513087895]"
511,78377.776581,6.119526e+06,218.312,"[170, 19, 58.65049351316202]"
702,78785.327402,6.119533e+06,407.619,"[131, 59, 39]"


In [250]:
#calc that box
trav1.pointList = trav1.pointList[0:4]

#509
trav1.add_point(angle =  dms(d([180,0,0])+ d([43,56,42])), distance = 8.484, name = "509", north = "given")

#510
trav1.add_point(angle =  [358-180,56,14], distance = 109.271, name = "510", north = "given")

#508
trav1.add_point(angle =  dms(d([180,0,0]) + d([90,10,58])), distance = 23.255, name = "508", north = "given")

#506
trav1.add_point(angle =  dms(d([180,0,0]) + d([43,56,39])), distance = 8.484, name = "506", north = "given")

#449
trav1.add_point(angle =  dms(d([180,0,0]) + d([88,57,3])), distance = 59.618, name = "449", north = "given")

trav1.print_points()

,Easting,Northing,Distance,Angle
Point,,,,
ASCM 35782,77761.853000,6.118936e+06,0,0
ASCM 62596,77958.809000,6.118907e+06,0,0
50,78234.727557,6.119361e+06,531.486,"[246, 59, 27.650493513087895]"
511,78377.776581,6.119526e+06,218.312,"[170, 19, 58.65049351316202]"
509,78371.888960,6.119520e+06,8.484,"[223, 56, 41.999999999975444]"
510,78373.915709,6.119411e+06,109.271,"[178, 56, 14]"
508,78350.660827,6.119411e+06,23.255,"[270, 10, 58.00000000008367]"
506,78344.773295,6.119405e+06,8.484,"[223, 56, 38.99999999997817]"
449,78285.165290,6.119404e+06,59.618,"[268, 57, 3.0000000001609806]"


In [251]:
#calc that box
#trav1.pointList = trav1.pointList[0:8]

#260
trav1.add_point(angle =  [359,1,50], distance = 64.5, name = "260", north = "given")

#100
trav1.add_point(angle =  [88,57,3], distance = 65.513, name = "100", north = "given")

trav1.print_points()

,Easting,Northing,Distance,Angle
Point,,,,
ASCM 35782,77761.853000,6.118936e+06,0,0
ASCM 62596,77958.809000,6.118907e+06,0,0
50,78234.727557,6.119361e+06,531.486,"[246, 59, 27.650493513087895]"
511,78377.776581,6.119526e+06,218.312,"[170, 19, 58.65049351316202]"
509,78371.888960,6.119520e+06,8.484,"[223, 56, 41.999999999975444]"
510,78373.915709,6.119411e+06,109.271,"[178, 56, 14]"
508,78350.660827,6.119411e+06,23.255,"[270, 10, 58.00000000008367]"
506,78344.773295,6.119405e+06,8.484,"[223, 56, 38.99999999997817]"
449,78285.165290,6.119404e+06,59.618,"[268, 57, 3.0000000001609806]"


In [241]:
a = []
type(a)

list